In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [23]:
scraped_data = pd.read_csv(r'..\..\data\raw\AnimeData_300724.csv')
scraped_data.columns = scraped_data.columns.str.lower().str.replace(' ', '_')
scraped_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26794 entries, 0 to 26793
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         26794 non-null  object 
 1   episodes      13552 non-null  float64
 2   release_year  26794 non-null  int64  
 3   status        26389 non-null  object 
 4   genres        21414 non-null  object 
 5   studio        25663 non-null  object 
 6   source        23896 non-null  object 
 7   demographic   10142 non-null  object 
 8   themes        0 non-null      float64
 9   synopsis      26785 non-null  object 
 10  voters        13078 non-null  float64
 11  rating        646 non-null    float64
dtypes: float64(4), int64(1), object(7)
memory usage: 2.5+ MB


In [24]:
impute_data = pd.read_csv(r"..\..\data\raw\anime-dataset-2023.csv")
impute_data.columns = impute_data.columns.str.lower().str.replace(' ', '_')
impute_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   anime_id      24905 non-null  int64 
 1   name          24905 non-null  object
 2   english_name  24905 non-null  object
 3   other_name    24905 non-null  object
 4   score         24905 non-null  object
 5   genres        24905 non-null  object
 6   synopsis      24905 non-null  object
 7   type          24905 non-null  object
 8   episodes      24905 non-null  object
 9   aired         24905 non-null  object
 10  premiered     24905 non-null  object
 11  status        24905 non-null  object
 12  producers     24905 non-null  object
 13  licensors     24905 non-null  object
 14  studios       24905 non-null  object
 15  source        24905 non-null  object
 16  duration      24905 non-null  object
 17  rating        24905 non-null  object
 18  rank          24905 non-null  object
 19  popu

In [25]:
scraped_columns = set(scraped_data.columns)
impute_columns = set(impute_data.columns)
scraped_columns.intersection(impute_columns)

{'episodes', 'genres', 'rating', 'source', 'status', 'synopsis'}

In [27]:
scraped_data.nunique()

title           26757
episodes          251
release_year      106
status              2
genres            966
studio           1026
source             16
demographic         5
themes              0
synopsis        21732
voters            971
rating             92
dtype: int64

In [28]:
impute_data.nunique()

anime_id        24905
name            24901
english_name    10134
other_name      23796
score             567
genres           1006
synopsis        20113
type                7
episodes          252
aired           15213
premiered         244
status              3
producers        4423
licensors         265
studios          1547
source             17
duration          331
rating              7
rank            15198
popularity      18363
favorites        1814
scored_by        8281
members         10996
image_url       24720
dtype: int64

In [33]:
scraped_data['synopsis'].value_counts()[:2]

synopsis
(No synopsis yet.)                                                                               4772
No synopsis has been added for this series yet.\r\n\r\nClick here to update this information.      25
Name: count, dtype: int64

In [35]:
scraped_data = scraped_data.replace(
    '(No synopsis yet.)', np.nan).replace(
        'No synopsis has been added for this series yet.\r\n\r\nClick here to update this information.', np.nan)

# Logic: Merge Scraped and Impute

List of common columns: {'episodes', 'genres', 'rating', 'source', 'status', 'synopsis'}

- Episode: pick the larger one.
- Genre: concatenate both of them
- Rating: pick the larger one
- Source: Pick from Impute
- Status: If available pick from Merge or else impute from Impute
- Synopsis: Concatenate both